In [ ]:
pip install rasterio

In [ ]:
pip install rioxarray

In [ ]:
pip install xarray-spatial

In [ ]:
pip install pystac-client

In [ ]:
pip install planetary-computer

In [ ]:
pip install odc-stac

In [ ]:
pip install odc-algo

In [15]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rasterio.features
import rioxarray as rio
import xrspatial.multispectral as ms

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
pc.settings.set_subscription_key('99d0028b4d864147958f0b6b44663e5f')
#import odc
from odc.stac import stac_load
from odc.algo import to_rgba
from tqdm import tqdm

# For finetuning ResNet-18
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
crop_presence_data = pd.read_csv("/content/drive/MyDrive/Crop_Location_Data_20221201.csv")

In [22]:
crop_presence_data["Latitude"] = crop_presence_data["Latitude and Longitude"].apply(lambda x: float(x[1:-1].split(",")[0]))
crop_presence_data["Longitude"] = crop_presence_data["Latitude and Longitude"].apply(lambda x: float(x[1:-1].split(",")[1]))

In [23]:
crop_presence_data.head()

,Latitude and Longitude,Class of Land,Latitude,Longitude
0,"(10.323727047081501, 105.2516346045924)",Rice,10.323727,105.251635
1,"(10.322364360592521, 105.27843410554115)",Rice,10.322364,105.278434
2,"(10.321455902933202, 105.25254306225168)",Rice,10.321456,105.252543
3,"(10.324181275911162, 105.25118037576274)",Rice,10.324181,105.251180
4,"(10.324635504740822, 105.27389181724476)",Rice,10.324636,105.273892


In [24]:
box_size_deg = 0.10
resolution = 20  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for CRS:4326 
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [25]:
def get_data_latlong(lat, long, time_window = "2020-03-20/2020-03-21"):
    min_lon = long-box_size_deg/2
    min_lat = lat-box_size_deg/2
    max_lon = long+box_size_deg/2
    max_lat = lat+box_size_deg/2
    bounds = (min_lon, min_lat, max_lon, max_lat)
    search = stac.search(collections=["sentinel-2-l2a"], bbox=bounds, datetime=time_window)
    items = list(search.get_all_items())
    xx = stac_load(
        items,
        bands=["red", "green", "blue", "nir", "SCL"],
        crs="EPSG:4326", # Latitude-Longitude
        resolution=scale, # Degrees
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=pc.sign,
        bbox=bounds
    )
    return xx

In [26]:
lat_arr = crop_presence_data["Latitude"]
long_arr = crop_presence_data["Longitude"]

In [27]:
xx_lst_1 = [0]*len(crop_presence_data)
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
for inx in tqdm(range(len(crop_presence_data)), desc = "Running"):
    xx = get_data_latlong(lat_arr[inx], long_arr[inx])
    xx_lst_1[inx] = xx

Running: 100%|██████████| 600/600 [02:14<00:00,  4.46it/s]


In [28]:
xx_lst_2 = [0]*len(crop_presence_data)
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
for inx in tqdm(range(len(crop_presence_data)), desc = "Running"):
    xx = get_data_latlong(lat_arr[inx], long_arr[inx], "2021-03-20/2021-03-21")
    xx_lst_2[inx] = xx

Running: 100%|██████████| 600/600 [02:13<00:00,  4.49it/s]


In [29]:
xx_lst_np_11 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_1[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_11[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:22<00:00,  1.40it/s]


In [30]:
xx_lst_np_12 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_2[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_12[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:35<00:00,  1.28it/s]


In [31]:
xx_lst_np_13 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_1[400+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_13[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:05<00:00,  1.60it/s]


In [32]:
xx_lst_np_21 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_2[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_21[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:01<00:00,  1.64it/s]


In [33]:
xx_lst_np_22 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_2[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_22[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [02:09<00:00,  1.55it/s]


In [34]:
xx_lst_np_23 = [0] * 200
for inx in tqdm(range(200), desc = "Running"):
    temp = xx_lst_2[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_23[inx] = temp.data.compute()

Running: 100%|██████████| 200/200 [01:56<00:00,  1.72it/s]


In [ ]:
for inx in range(200):
    xx_lst_np_11[inx] = np.resize(xx_lst_np_11[inx], (3, 557, 557))
    xx_lst_np_12[inx] = np.resize(xx_lst_np_12[inx], (3, 557, 557))
    xx_lst_np_13[inx] = np.resize(xx_lst_np_13[inx], (3, 557, 557))

In [ ]:
data = np.concatenate([np.array(xx_lst_np_11), np.array(xx_lst_np_12), np.array(xx_lst_np_13)], axis = 0)
# 4D array used for ResNet-18

In [ ]:
data.shape

In [35]:
crop_presence_data["Class"] = crop_presence_data["Class of Land"].apply(lambda x: 1 if x == "Rice" else 0)

In [37]:
# Now convert to 2D array to use DT
for inx in range(200):
    xx_lst_np_11[inx] = xx_lst_np_11[inx].reshape(3*557*557)
    xx_lst_np_12[inx] = xx_lst_np_12[inx].reshape(3*557*557)
    xx_lst_np_13[inx] = xx_lst_np_13[inx].reshape(3*557*557)
    """
    xx_lst_np_21[inx] = np.resize(xx_lst_np_21[inx], (3, 557, 557)).reshape(3*557*557)
    xx_lst_np_22[inx] = np.resize(xx_lst_np_22[inx], (3, 557, 557)).reshape(3*557*557)
    xx_lst_np_23[inx] = np.resize(xx_lst_np_23[inx], (3, 557, 557)).reshape(3*557*557)
    """

In [38]:
x = np.concatenate([np.array(xx_lst_np_11), np.array(xx_lst_np_12), np.array(xx_lst_np_13),\
                    np.array(xx_lst_np_21), np.array(xx_lst_np_22), np.array(xx_lst_np_23)],\
                   axis = 0)

In [39]:
y = np.concatenate([np.array(crop_presence_data["Class"]), np.array(crop_presence_data["Class"])])

In [41]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 101)

In [42]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

In [43]:
clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [44]:
pred = clf.predict(x_test)

In [45]:
from sklearn.metrics import classification_report

In [46]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.64      0.78      0.70       166
           1       0.77      0.62      0.69       194

    accuracy                           0.69       360
   macro avg       0.70      0.70      0.69       360
weighted avg       0.71      0.69      0.69       360



In [ ]:
# Load the ResNet-18 model
resnet = models.resnet18(pretrained=True)

# Modify the last layer
num_classes = 2
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

# Prepare the input data
# Assume the input data is a 3D numpy array with shape (num_samples, depth, height, width)
mean = np.mean(data)
std = np.std(data)
data = (data - mean) / std

# Prepare the labels
labels = y
 # prepare labels in the appropriate format

# Convert the input data and labels to PyTorch tensors
input_data = torch.from_numpy(data.astype("uint8")).float()
labels = torch.from_numpy(labels).long()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
batch_size = 32
for epoch in range(num_epochs):
    for i in range(0, len(input_data), batch_size):
        inputs = input_data[i:i+batch_size]
        target = labels[i:i+batch_size]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = resnet(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # Print statistics
        if i % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(input_data), loss.item()))

In [47]:
test_data = pd.read_csv("/content/drive/MyDrive/challenge_1_submission_template_correct_columns_fixed.csv")

In [48]:
test_data.head()

,id,target
0,"(10.18019073690894, 105.32022315786804)",NaN
1,"(10.561107033461816, 105.12772097986661)",NaN
2,"(10.623790611954897, 105.13771401411867)",NaN
3,"(10.583364246115156, 105.23946127195805)",NaN
4,"(10.20744446668854, 105.26844107128906)",NaN


In [49]:
test_data["Latitude"] = test_data["id"].apply(lambda x: float(x[1:-1].split(",")[0]))
test_data["Longitude"] = test_data["id"].apply(lambda x: float(x[1:-1].split(",")[1]))

In [50]:
test_data.head()

,id,target,Latitude,Longitude
0,"(10.18019073690894, 105.32022315786804)",NaN,10.180191,105.320223
1,"(10.561107033461816, 105.12772097986661)",NaN,10.561107,105.127721
2,"(10.623790611954897, 105.13771401411867)",NaN,10.623791,105.137714
3,"(10.583364246115156, 105.23946127195805)",NaN,10.583364,105.239461
4,"(10.20744446668854, 105.26844107128906)",NaN,10.207444,105.268441


In [67]:
test_lst = [0]*250
lat_arr_test = test_data["Latitude"]
long_arr_test = test_data["Longitude"]
for inx in tqdm(range(250), desc="Running"):
    xx = get_data_latlong(lat_arr_test[inx], long_arr_test[inx], "2022-03-20/2022-03-21")
    test_lst[inx] = xx

Running: 100%|██████████| 250/250 [00:58<00:00,  4.29it/s]


In [68]:
test_lst_np_1 = [0] * 125
for inx in tqdm(range(125), desc="Running"):
    temp = test_lst[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_1[inx] = temp.data.compute()

Running: 100%|██████████| 125/125 [01:28<00:00,  1.41it/s]


In [ ]:
test_lst_np_2 = [0] * 125
for inx in tqdm(range(125), desc="Running"):
    temp = test_lst[125+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_2[inx] = temp.data.compute()

Running:  46%|████▌     | 57/125 [01:04<01:25,  1.26s/it]

In [ ]:
"""
test_lst_np_1 = test_lst_np_1[0:125]
test_lst_np_2 = test_lst_np_2[0:125]
"""

In [ ]:
for inx in range(125):
    test_lst_np_1[inx] = np.resize(test_lst_np_1[inx], (3, 557, 557)).reshape(3*557*557)
    test_lst_np_2[inx] = np.resize(test_lst_np_2[inx], (3, 557, 557)).reshape(3*557*557)

In [ ]:
my_test = np.concatenate([np.array(test_lst_np_1), np.array(test_lst_np_2)], axis = 0)

In [ ]:
pred_test = clf.predict(my_test)

In [ ]:
len(pred_test)

In [ ]:
test_data["target"] = pd.DataFrame({"target": pred_test})

In [ ]:
test_data.head()

In [ ]:
export = test_data[["id", "target"]]

In [ ]:
export["target"] = export["target"].apply(lambda x: "Rice" if 1 else "Non Rice")

In [ ]:
export.to_csv("/content/drive/MyDrive/Test_result_colab.csv", index = False)